In [ ]:
import numpy as np

#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.tri as tri

from sklearn.decomposition import PCA

from astropy import units as u
from astropy.io import ascii

from mmtwfs.zernike import ZernikeVector

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
ACT = 104 # number of active actuators
NODE = 3222 # number of FE nodes in BCV analysis
BRAD = 3228.5 # radius of M1 in mm

In [ ]:
# matrix to convert a phase surface calculated at each BCV node into a vector of actuator force commands
# this version uses the first 32 modes from an SVD analysis.
# surface units are nm, actuator force commands are in Newtons
surf2act = np.fromfile("../mmtwfs/data/Surf2ActTEL_32.bin", dtype=np.float32).reshape(ACT, NODE).T

# coordinates of the BCV FE nodes
nodecoor = ascii.read(
    "../mmtwfs/data/bcv_node_coordinates.dat",
    names=["bcv_id", "bcv_x", "bcv_y", "bcv_z"]
)

# coordinates of the actuators
actcoor = ascii.read(
    "../mmtwfs/data/actuator_coordinates.dat",
    names=["act_i", "act_x", "act_y", "act_type"]
)

# add index to be able to query by actuator number
actcoor.add_index("act_i")

# convert BCV coordinates to a unit circle
for ax in ["bcv_x", "bcv_y"]:
    nodecoor[ax] /= BRAD

# calculate polar coordinates for the BCV nodes
nodecoor['bcv_rho'] = np.sqrt(nodecoor['bcv_x']**2 + nodecoor['bcv_y']**2)
nodecoor['bcv_phi'] = np.arctan2(nodecoor['bcv_y'], nodecoor['bcv_x'])

# full 104 mode matrix to convert a phase surface calculated at each BCV node
# into a vector of actuator force commands
surf2act_full = np.loadtxt("../mmtwfs/data/Surf2ActTEL_104").T

# matrix to convert a force vector in unit of Newtons into a phase surface
# at each BCV node in units of nm
act2surf = np.loadtxt("../mmtwfs/data/Act2SurfTEL").T

In [ ]:
def plot_actuator_influence(act_id, force=1.0):
    """
    Plot the phase surface created by applying <force> Newtons to actuator `act_id`.
    """
    actuator_info = actcoor.loc[act_id]
    actuator = actuator_info['act_i']
    # this maps act_id to force vector index
    act_index = actcoor['act_i'].searchsorted(actuator)
    fig, ax = plt.subplots()
    fig.set_label(f"Actuator ID: {actuator}")
    fvec = np.zeros(ACT)
    fvec[act_index] = force
    ph = fvec @ act2surf
    X, Y = nodecoor['bcv_x'].value, nodecoor['bcv_y'].value
    triang = tri.Triangulation(X, Y)
    # Mask off hole in center of mirror
    triang.set_mask(
        np.hypot(
            X[triang.triangles].mean(axis=1),
            Y[triang.triangles].mean(axis=1)
        ) < np.min(nodecoor['bcv_rho'])
    )
    ax.set_aspect('equal')
    vmax = np.ceil(np.max(np.abs(ph))+1)
    vmin = -vmax+1
    levels = np.arange(vmin, vmax)
    tcf = ax.tricontourf(triang, ph, vmax=vmax, vmin=vmin, levels=levels, cmap=cm.RdBu)
    ax.set_axis_off()
    fig.colorbar(tcf, ax=ax, label='Wavefront Error (nm)')
    ax.tricontour(triang, ph, vmax=vmax, vmin=vmin, levels=levels, colors='k')
    xcor = actcoor['act_x']/BRAD
    ycor = actcoor['act_y']/BRAD
    ax.scatter(xcor, ycor, marker='.', color='black')
    for i, (x, y) in enumerate(zip(xcor, ycor)):
        ax.text(
            x,
            y + 0.02,
            actcoor["act_i"][i],
            horizontalalignment="center",
            verticalalignment="bottom",
            size="xx-small",
            color="black"
        )
    ax.scatter(actuator_info['act_x']/BRAD, actuator_info['act_y']/BRAD, color='lime', marker='*')
    return fig

def zernike_surface(zv=ZernikeVector()):
    """
    Calculate phase surface at the BCV node positions for a given ZernikeVector `zv`.
    Displacements are in nm.
    """
    # convert to nm...
    zv.units = u.nm

    # make sure we're not Noll normalized...
    zv.denormalize()

    # need to rotate the wavefront -90 degrees to match the BCV angle convention of +Y being 0 deg.
    zv.rotate(-90 * u.deg)

    # get surface displacements at the BCV node positions. multiply the wavefront amplitude by 0.5 to account for
    # reflection off the surface.
    surface = (
        -0.5 * zv.total_phase(nodecoor["bcv_rho"], nodecoor["bcv_phi"])
    )

    return surface.to(u.nm).value

def plot_phase_surface(zsurf):
    """
    Plot a phase surface <zsurf> that was calculated at the BCV node positions.
    """
    fig, ax = plt.subplots()
    fig.set_label("Phase Surface")
    X, Y = nodecoor['bcv_x'].value, nodecoor['bcv_y'].value
    triang = tri.Triangulation(X, Y)
    # Mask off hole in center of mirror
    triang.set_mask(
        np.hypot(
            X[triang.triangles].mean(axis=1),
            Y[triang.triangles].mean(axis=1)
        ) < np.min(nodecoor['bcv_rho'])
    )
    ax.set_aspect('equal')
    tcf = ax.tricontourf(triang, zsurf, cmap=cm.RdBu)
    ax.set_axis_off()
    fig.colorbar(tcf, ax=ax, label='Wavefront Error (nm)')
    ax.tricontour(triang, zsurf, colors='k')
    return fig

In [ ]:
fig = plot_actuator_influence(43, force=1)


In [ ]:
zv = ZernikeVector(Z05=400*u.nm)
zsurf = zernike_surface(zv)
fig = plot_phase_surface(zsurf)

In [ ]:
# create matrix of surfaces for all actuators
act_surfaces = np.identity(ACT) @ act2surf
# perform PCA on the actuator surfaces to generate a reduced basis of bending modes
pca = PCA(n_components=13).fit(act_surfaces)

In [ ]:
vr = np.cumsum(pca.explained_variance_ratio_)
plt.figure()
plt.plot(vr)
plt.xlabel('number of components')
plt.ylabel('explained variance')
plt.show()
vr[-1]

In [ ]:
components = pca.transform([zsurf])
filtered = pca.inverse_transform(components)
resid = zsurf - filtered
fig = plot_phase_surface(resid[0])
#fig = plot_phase_surface(zsurf)

In [ ]:
fig = plot_phase_surface(pca.components_[0] / pca.mean_[0])

In [ ]:
# try to create a new surf2act via moore-penrose pseudoinverse. basically an SVD, but more user-friendly.
inv = np.linalg.pinv(act2surf, rcond=0.005)  # Use rcond reject less significant modes
inv

In [ ]:
# check quality of the fitted forces using the pseudoinverse matrix
inv_act = zsurf @ inv
inv_resid = zsurf - inv_act @ act2surf
fig = plot_phase_surface(inv_resid)

In [ ]:
# check quality of the fitted forces using the original 32-mode SVD matrix
svd_act = zsurf @ surf2act
svd_resid = zsurf - svd_act @ act2surf
fig = plot_phase_surface(svd_resid)

In [ ]:
# max required force is similar for both so the pseudoinverse
# is a good reconstruction of what we have been using
np.abs(inv_act).max(), np.abs(svd_act).max()

In [ ]:
buddy_forces = [
    -22.5, -25.8, -15.3, 65.5, 48.8, -33.6, 51.4, 100.7, 126.6, 17.4, 24.5, 120.2, 135.7, -27.5, 8.9,
	-13.8, 63.4, 85.9, -8.9, 86.4, -35.8, 65.8, 92.9, 34.0, 47.8, 8.6, 71.7, 75.2, 38.1, 46.3,
	21.5, 29.5, 8.3, 2.8, 52.1, 24.7, -6.0, 10.4, 55.3, 40.9, 38.0, 43.8, 51.4, 49.5, 67.1,
	75.6, 62.9, 51.2, 55.5, -10.2, 13.0, 32.4, -10.8, -14.5, -11.5, 75.1,
    52.7, -11.9, 52.4, 94.2, 117.4, 49.8, 22.7, 101.8, 134.3, 31.3, 9.3,
	-28.5, 27.1, 72.7, 16.9, 59.9, -88.8, 10.3, 60.3, 9.3, 13.9, -8.7, 60.9, 60.3, 50.1, 98.4,
	86.8, 66.0, 16.4, 8.8, 94.4, 93.0, 40.1, 20.4, 47.7, 42.1, 35.9, 12.3, 42.7, 25.4, 43.6,
	30.5, 55.8, 43.5, 15.0, -11.2, 13.7, 23.6
]

In [ ]:
buddy_surf = buddy_forces @ act2surf
fig = plot_phase_surface(buddy_surf)

In [ ]:
np.min(buddy_forces), np.max(buddy_forces), np.mean(buddy_forces), np.std(buddy_forces)